In [2]:
import scipp as sc
import numpy as np
import matplotlib.pyplot as plt
import plopp as pp
%matplotlib widget
from scipp import curve_fit
from scipy.special import voigt_profile



In [20]:
## Load and inspect data
#TODO Make a general data loader
## Load actual data - just one temperature for now
NUMBER_OF_Q_POINTS=16
NUMBER_OF_E_POINTS=1024
T_values=[0.060, 0.175, 0.450, 0.600, 1]

#Preallocate
intensity_values=np.zeros((NUMBER_OF_Q_POINTS,NUMBER_OF_E_POINTS,len(T_values)))
error_values=np.zeros((NUMBER_OF_Q_POINTS,NUMBER_OF_E_POINTS,len(T_values)))


# Load data into a matrix
for T_index in range(len(T_values)):
    for Q_index in range(NUMBER_OF_Q_POINTS):
        filename = '../../IN16b_GGG_data/data_' + str(round(T_values[T_index]*1e3))  +'mK_Q' +str(Q_index+1) +'.dat'

        data_array = np.loadtxt(filename)
        energy_values=data_array[:, 0] #should be the same for all Q
        # EnergyValues[Q_index,:]=data_array[:, 0]
        intensity_values[Q_index,:,T_index]=data_array[:,1]
        error_values[Q_index,:,T_index]=data_array[:,2]

# Define energy, q and intensity as scipp variables with units, and make a DataArraw
energy=sc.array(dims=['energy'],values=energy_values/1000,unit='meV')
Q=sc.array(dims=['Q'],values=range(NUMBER_OF_Q_POINTS))
temperature=sc.array(dims=['temperature'],values=T_values,unit='K')
intensity=sc.array(dims=['Q','energy','temperature'],values=intensity_values,variances=error_values*error_values) #The variance is the square of the uncertainty!

ENERGY_BINS=sc.scalar(1e-3*0.2, unit='meV')

GGG_data = sc.DataArray(data=intensity, coords={'Q':Q,'energy': energy, 'temperature':temperature})
binned_GGG_data=GGG_data.flatten(to='dummy').bin(energy=ENERGY_BINS,Q=Q,temperature=temperature).bins.mean() #can add .plot() to plot it



<scipp.Variable> (Q: 16)      int32  [dimensionless]  [0, 1, ..., 14, 15]

In [22]:
INTENSITY_MIN=0.0
INTENSITY_MAX=0.001

ENERGY_MIN = -0.03 * sc.Unit('meV')
ENERGY_MAX = 0.03 * sc.Unit('meV')

pp.slicer(binned_GGG_data['energy',ENERGY_MIN:ENERGY_MAX],vmin=INTENSITY_MIN,vmax=INTENSITY_MAX,
     keep=['energy'],
     linestyle='none',
     marker='o',
     markerfacecolor='none',
     color='k'
)

Box(children=(InteractiveFig(children=(HBar(), HBox(children=(VBar(children=(Toolbar(children=(ButtonTool(icon…